# Imports

In [ ]:
import pandas as pd

In [ ]:
import holoviews as hv
from holoviews import opts
from holoviews.streams import Stream, param
import panel as pn
from bokeh.models.formatters import PrintfTickFormatter
from bokeh.models.renderers import GlyphRenderer
from bokeh.models import Range1d, LinearAxis
from bokeh.models import HoverTool

# is needed for unlocking bokeh doc
from panel.io import unlocked

pn.extension('katex')
hv.extension('bokeh', width=100)

# pn.config.sizing_mode = 'stretch_both'

In [ ]:
from oggm.core.massbalance import MassBalanceModel
from oggm import cfg
from oggm.cfg import SEC_IN_YEAR, SEC_IN_MONTH

cfg.initialize_minimal()

In [ ]:
import numpy as np
from functools import partial

# Define Domain

In [ ]:
# glacier top height
max_height = 6000

# glacier bottom height
min_height = 0

# Define default base climates

Values from www.climate-data.org

In [ ]:
base_climate_names = ['Mid Latitudes (Innsbruck)',
                      'Tropics (Nairobi)']
base_climate_temps = [np.array([6, 0.1, -4.6, -6, -4.1, -0.1, 3.7, 8.4, 12.6, 14.2, 14, 10]),
                      np.array([19.3, 18.3, 18.6, 19.7, 20.5, 20.5, 19.3, 18.5, 17.6, 17, 17.3, 18.5])]
base_climate_prcps = [np.array([120., 114., 99., 88., 86., 116., 124., 182., 202., 210., 208., 144.]),
                      np.array([49., 94., 55., 37., 34., 62., 125., 102., 48., 20., 28., 20.])]
base_climate_ref_hgts = [584,
                         1669]

# Define Menu

In [ ]:
# define background color of tab menu
menu_background = '#f4f4f4'

# define width for menu-tabs
panel_width = 250

# define width for whole menu
tab_menu_width = 370

# needed for some initialisations for start of app
initial_run = True

## Climate Tab

In [ ]:
# Climate Tab
def set_default_base_climate(event):
    if event.new == base_climate_names[0]:
        ref_height_slider.value = int(base_climate_ref_hgts[0])
    elif event.new == base_climate_names[1]:
        ref_height_slider.value = int(base_climate_ref_hgts[1])

    temp_bias_slider.value = 0.
    prcp_fac_slider.value = cfg.PARAMS['prcp_scaling_factor']
    temp_grad_slider.value = cfg.PARAMS['temp_default_gradient'] * 100
    prcp_grad_slider.value = 0.
base_climate_select = pn.widgets.Select(name='Base Climate',
                                        options=base_climate_names)
base_climate_select.param.watch(set_default_base_climate, ['value'])

ref_height_slider = pn.widgets.IntSlider(name='Reference Height',
                                         start=min_height,
                                         end=max_height,
                                         step=1,
                                         value=base_climate_ref_hgts[0])

temp_bias_slider = pn.widgets.FloatSlider(name='Temperatur Bias',
                                          start=-2,
                                          end=2,
                                          step=0.1,
                                          value=0)

prcp_fac_slider = pn.widgets.FloatSlider(name='Precipitation factor',
                                         start=0.5,
                                         end=5,
                                         step=0.1,
                                         value=cfg.PARAMS['prcp_scaling_factor'])

temp_grad_slider = pn.widgets.FloatSlider(name='Temperatur Gradient',
                                          start=cfg.PARAMS['temp_local_gradient_bounds'][0] * 100,
                                          end=cfg.PARAMS['temp_local_gradient_bounds'][1] * 100,
                                          step=0.01,
                                          value=cfg.PARAMS['temp_default_gradient'] * 100)
temp_grad_slider.format = PrintfTickFormatter(format='%.2f °C/100m')

prcp_grad_slider = pn.widgets.FloatSlider(name='Precipitation Gradient',
                                          start=-5,
                                          end=5,
                                          step=0.1,
                                          value=0)
prcp_grad_slider.format = PrintfTickFormatter(format='%.1f mm / m')

#busy_indicator_climate_tab = pn.indicators.LoadingSpinner(width=30,
#                                                          height=30,
#                                                          color='light',
#                                                          bgcolor='dark')


def set_climate_button_click(event):
    #toogle_work_status()
    # toogle_busy_indicators()
    # toogle_buttons_dsiabled()
    set_climate_of_mb_model(current_mb_i)
    update_current_figures(current_mb_i)
    # toogle_buttons_dsiabled()
    # toogle_busy_indicators()
    #toogle_work_status()


set_climate_button = pn.widgets.Button(name='Set Climate',
                                       sizing_mode='stretch_width')
set_climate_button.on_click(set_climate_button_click)

climate_tab = pn.Column(base_climate_select,
                        ref_height_slider,
                        temp_bias_slider,
                        prcp_fac_slider,
                        temp_grad_slider,
                        prcp_grad_slider,
                        set_climate_button,
                        background=menu_background,
                        width=panel_width,
                        sizing_mode='stretch_height')

## MB Settings

In [ ]:
# MB Settings
mu_slider = pn.widgets.IntSlider(name='Sensitivity parameter µ',
                                 start=100,
                                 end=300,
                                 step=10,
                                 value=200)

temp_melt_slider = pn.widgets.FloatSlider(name='T for ice melt (T_melt)',
                                          start=-2,
                                          end=0,
                                          step=0.1,
                                          value=cfg.PARAMS['temp_melt'])

temp_solid_slider = pn.widgets.FloatSlider(name='T for solid Prcp only (T_solid)',
                                           start=-0.5,
                                           end=0.5,
                                           step=0.1,
                                           value=cfg.PARAMS['temp_all_solid'])

temp_liquid_slider = pn.widgets.FloatSlider(name='T for liquid Prcp only (T_liquid)',
                                            start=0.5,
                                            end=3,
                                            step=0.1,
                                            value=cfg.PARAMS['temp_all_liq'])


def set_mb_settings_click(event):
    # toogle_work_status()
    # toogle_busy_indicators()
    # toogle_buttons_dsiabled()
    set_mb_settings_of_mb_model(current_mb_i)
    update_current_figures(current_mb_i)
    # toogle_buttons_dsiabled()
    # toogle_busy_indicators()
    # toogle_work_status()


set_mb_settings_button = pn.widgets.Button(name='Set MB Settings',
                                           sizing_mode='stretch_width')
set_mb_settings_button.on_click(set_mb_settings_click)

#busy_indicator_mb_settings_tab = pn.indicators.LoadingSpinner(width=30,
#                                                              height=30,
#                                                              color='light',
#                                                              bgcolor='dark')

MB_settings_tab = pn.Column(mu_slider,
                            temp_melt_slider,
                            temp_solid_slider,
                            temp_liquid_slider,
                            set_mb_settings_button,
                            background=menu_background,
                            width=panel_width,
                            sizing_mode='stretch_height')

## Custom Climate

In [ ]:
custom_climate_tab = pn.Column(pn.pane.Markdown(object=('<p style="margin-top: 0px;">' +
                                                        'TODO: Possibility to define your own climate'
                                                        + '</p>'),
                                                # height=20,
                                                margin=(0, 0),
                                                align='end',),
                               background=menu_background,
                               width=panel_width,
                               sizing_mode='stretch_height'
                               )

## Toogle busy indicators

In [ ]:
def toogle_busy_indicators():
    busy_indicator_climate_tab.value = not busy_indicator_climate_tab.value
    busy_indicator_mb_settings_tab.value = not busy_indicator_mb_settings_tab.value

## Toogle buttons disabled

In [ ]:
def toogle_buttons_dsiabled():
    set_mb_settings_button.disabled = not set_mb_settings_button.disabled
    set_climate_button.disabled = not set_climate_button.disabled

## But all Tabs together

In [ ]:
tab_menu = pn.Tabs(('Climate', climate_tab),
                   ('MB Settings', MB_settings_tab),
                   ('Custom Climate', custom_climate_tab),
                   height=420,
                   width=tab_menu_width,
                   background=menu_background,
                   tabs_location='left')

## functions to enable and disable menu

In [ ]:
def toogle_disabled_of_menu():
    base_climate_select.disabled = not base_climate_select.disabled
    ref_height_slider.disabled = not ref_height_slider.disabled
    temp_bias_slider.disabled = not temp_bias_slider.disabled
    prcp_fac_slider.disabled = not prcp_fac_slider.disabled
    temp_grad_slider.disabled = not temp_grad_slider.disabled
    prcp_grad_slider.disabled = not prcp_grad_slider.disabled
    set_climate_button.disabled = not set_climate_button.disabled
    mu_slider.disabled = not mu_slider.disabled
    temp_melt_slider.disabled = not temp_melt_slider.disabled
    temp_solid_slider.disabled = not temp_solid_slider.disabled
    temp_liquid_slider.disabled = not temp_liquid_slider.disabled
    set_mb_settings_button.disabled = not set_mb_settings_button.disabled

# Define function for text

In [ ]:
def get_simple_text(text='', width=30, font_size='30pt'):
    return pn.Column(pn.layout.VSpacer(),
                     pn.widgets.StaticText(value=text,
                                           style={'font-size': font_size}),
                     pn.layout.VSpacer(),
                     width=width,
                     sizing_mode="stretch_height",
                     )

In [ ]:
def get_latex_text(text='', width=50, font_size='40pt'):
    return pn.Column(pn.layout.VSpacer(),
                     pn.pane.LaTeX(text,
                                   style={'font-size': font_size}),
                     pn.layout.VSpacer(),
                     width=width,
                     sizing_mode="stretch_height",
                     )

# MassBalanceClass

adapted from OGGM PastMassBalance https://github.com/OGGM/oggm/blob/master/oggm/core/massbalance.py#L286

In [ ]:
class OGGMMassBalance(MassBalanceModel):
    """Mass balance during the climate data period."""

    def __init__(self, min_height, max_height,
                 temp,
                 prcp,
                 mb_model_nr=None,
                 temp_grad=None,
                 prcp_grad=None,
                 ref_hgt=None,
                 base_climate_name='',
                 mu=200,
                 t_solid=None,
                 t_liq=None,
                 t_melt=None,
                 prcp_fac=None,
                 repeat=False, ys=None, ye=None):
        """Initialize.
        Parameters
        ----------
        gdir : GlacierDirectory
            the glacier directory
        mu_star : float, optional
            set to the alternative value of mu* you want to use
            (the default is to use the calibrated value).
        bias : float, optional
            set to the alternative value of the calibration bias [mm we yr-1]
            you want to use (the default is to use the calibrated value)
            Note that this bias is *substracted* from the computed MB. Indeed:
            BIAS = MODEL_MB - REFERENCE_MB.
        filename : str, optional
            set to a different BASENAME if you want to use alternative climate
            data.
        input_filesuffix : str
            the file suffix of the input climate file
        repeat : bool
            Whether the climate period given by [ys, ye] should be repeated
            indefinitely in a circular way
        ys : int
            The start of the climate period where the MB model is valid
            (default: the period with available data)
        ye : int
            The end of the climate period where the MB model is valid
            (default: the period with available data)
        check_calib_params : bool
            OGGM will try hard not to use wrongly calibrated mu* by checking
            the parameters used during calibration and the ones you are
            using at run time. If they don't match, it will raise an error.
            Set to False to suppress this check.
        Attributes
        ----------
        temp_bias : float, default 0
            Add a temperature bias to the time series
        prcp_fac : float, default cfg.PARAMS['prcp_scaling_factor']
            Precipitation factor to the time series (called factor to make clear
             that it is a multiplicative factor in contrast to the additive
             `temp_bias`)
        """

        super(OGGMMassBalance, self).__init__()
        self.valid_bounds = [min_height, max_height]  # in m
        self.default_heights = np.linspace(self.valid_bounds[0],
                                           self.valid_bounds[1],
                                           num=50,
                                           endpoint=True)

        self.base_climate_name = base_climate_name
        self.mb_model_nr = mb_model_nr

        self.mu = np.array(mu)

        # Parameters
        if t_solid is None:
            self.t_solid = np.array(cfg.PARAMS['temp_all_solid'])
        else:
            self.t_solid = np.array(t_solid)
        if t_liq is None:
            self.t_liq = np.array(cfg.PARAMS['temp_all_liq'])
        else:
            self.t_liq = np.array(t_liq)
        if t_melt is None:
            self.t_melt = np.array(cfg.PARAMS['temp_melt'])
        else:
            self.t_melt = np.array(t_melt)
        if prcp_fac is None:
            prcp_fac = np.array(cfg.PARAMS['prcp_scaling_factor'])
        # check if valid prcp_fac is used
        if prcp_fac <= 0:
            raise InvalidParamsError('prcp_fac has to be above zero!')

        # Private attrs
        # to allow prcp_fac to be changed after instantiation
        # prescribe the prcp_fac as it is instantiated
        self._prcp_fac = np.array(prcp_fac)
        # same for temp bias
        self._temp_bias = np.array(0.)

        # using months in hydro year
        self.months = {'Oct': 0,
                       'Nov': 1,
                       'Dec': 2,
                       'Jan': 3,
                       'Feb': 4,
                       'Mar': 5,
                       'Apr': 6,
                       'May': 7,
                       'Jun': 8,
                       'Jul': 9,
                       'Aug': 10,
                       'Sep': 11}

        if ref_hgt is None:
            ref_hgt = np.array(self.valid_bounds[0])
        self.ref_hgt = np.array(ref_hgt)

        self.temp = np.array(temp) + self._temp_bias
        self.prcp = np.array(prcp) * self._prcp_fac

        if temp_grad is None:
            self.temp_grad = np.array(cfg.PARAMS['temp_default_gradient'])
        else:
            self.temp_grad = np.array(temp_grad)
        if prcp_grad is None:
            self.prcp_grad = np.array(0.)
        else:
            self.prcp_grad = np.array(prcp_grad)

        self.default_tools = ['xwheel_zoom', 'hover', 'save']
        self.active_tools = ['xwheel_zoom']

    # adds the possibility of changing prcp_fac
    # after instantiation with properly changing the prcp time series
    @property
    def prcp_fac(self):
        return self._prcp_fac

    @prcp_fac.setter
    def prcp_fac(self, new_prcp_fac):
        # just to check that no invalid prcp_factors are used
        if new_prcp_fac <= 0:
            raise InvalidParamsError('prcp_fac has to be above zero!')
        self.prcp *= new_prcp_fac / self._prcp_fac
        # update old prcp_fac in order that it can be updated again ...
        self._prcp_fac = new_prcp_fac

    # same for temp_bias:
    @property
    def temp_bias(self):
        return self._temp_bias

    @temp_bias.setter
    def temp_bias(self, new_temp_bias):
        self.temp += new_temp_bias - self._temp_bias
        # update old temp_bias in order that it can be updated again ...
        self._temp_bias = new_temp_bias

    def get_monthly_climate(self, heights=None, month=None):
        """Monthly climate information at given heights.
        Note that prcp is corrected with the precipitation factor and that
        all other model biases (temp and prcp) are applied.
        Returns
        -------
        (temp, tempformelt, prcp, prcpsol)
        """

        if month is None:
            pok = self.months['Oct']
        else:
            pok = self.months[month]

        # Read already (temperature bias and precipitation factor corrected!)
        itemp = self.temp[pok]
        iprcp = self.prcp[pok]

        if heights is None:
            heights = self.default_heights

        # For each height pixel:
        # Compute temp and tempformelt (temperature above melting threshold)
        npix = len(heights)
        temp = np.ones(npix) * itemp + self.temp_grad * \
            (heights - self.ref_hgt)
        tempformelt = temp - self.t_melt
        np.clip(tempformelt, 0, None, out=tempformelt)

        # Compute solid precipitation from total precipitation
        prcp_total = np.ones(npix) * iprcp + \
            self.prcp_grad * (heights - self.ref_hgt)
        fac = 1 - (temp - self.t_solid) / (self.t_liq - self.t_solid)
        np.clip(fac, 0, 1, out=fac)
        prcp_sol = prcp_total * fac

        return temp, tempformelt, prcp_total, prcp_sol, fac

    def _get_2d_annual_climate(self, heights=None):
        if heights is None:
            heights = self.default_heights

        # Read already (temperature bias and precipitation factor corrected!)
        itemp = self.temp
        iprcp = self.prcp

        # For each height pixel:
        # Compute temp and tempformelt (temperature above melting threshold)
        heights = np.asarray(heights)
        npix = len(heights)
        grad_temp = np.atleast_2d(self.temp_grad.repeat(12)).repeat(npix, 0)
        grad_temp *= (heights.repeat(12).reshape(grad_temp.shape) -
                      self.ref_hgt)
        temp2d = np.atleast_2d(itemp).repeat(npix, 0) + grad_temp
        temp2dformelt = temp2d - self.t_melt
        np.clip(temp2dformelt, 0, None, out=temp2dformelt)

        # Compute solid precipitation from total precipitation
        grad_prcp = np.atleast_2d(self.prcp_grad.repeat(12)).repeat(npix, 0)
        grad_prcp *= (heights.repeat(12).reshape(grad_prcp.shape) -
                      self.ref_hgt)
        prcp2dtotal = np.atleast_2d(iprcp).repeat(npix, 0) + grad_prcp
        fac2d = 1 - (temp2d - self.t_solid) / (self.t_liq - self.t_solid)
        np.clip(fac2d, 0, 1, out=fac2d)
        prcp2dsol = prcp2dtotal * fac2d

        return temp2d, temp2dformelt, prcp2dtotal, prcp2dsol, fac2d

    def get_annual_climate(self, heights=None):
        """Annual climate information at given heights.
        Note that prcp is corrected with the precipitation factor and that
        all other model biases (temp and prcp) are applied.
        Returns
        -------
        (temp, tempformelt, prcp, prcpsol)
        """
        t, tmelt, prcp, prcpsol, fac = self._get_2d_annual_climate(
            heights=heights)
        return (t.mean(axis=1), tmelt.sum(axis=1),
                prcp.sum(axis=1), prcpsol.sum(axis=1))

    def get_monthly_mb(self, heights=None, month=None, **kwargs):

        t, tformelt, prcp, prcpsol, fac = self.get_monthly_climate(
            heights=heights, month=month)
        mb_month = prcpsol - self.mu * tformelt
        return mb_month / SEC_IN_MONTH / self.rho

    def get_annual_mb(self, heights=None, **kwargs):

        t, tformelt, prcp, prcpsol, fac = self._get_2d_annual_climate(
            heights=heights)
        mb_annual = np.sum(prcpsol - self.mu * tformelt, axis=1)
        mb_annual = mb_annual / SEC_IN_YEAR / self.rho
        return mb_annual

    def get_climograph(self, ref_height=None, **kwargs):
        def prcp_hook(plot, element):
            p = plot.state

            # color yaxis of prcp axis
            prcp_color = 'blue'
            plot.handles['yaxis'].axis_label_text_color = prcp_color
            plot.handles['yaxis'].major_label_text_color = prcp_color
            plot.handles['yaxis'].axis_line_color = prcp_color
            plot.handles['yaxis'].major_tick_line_color = prcp_color
            plot.handles['yaxis'].minor_tick_line_color = prcp_color

            # set y_range of first axis
            #glyph = p.renderers[-1]
            #vals = glyph.data_source.data['Precipitation_left_parenthesis_mm_right_parenthesis']
            #plot.handles['y_range'].start = 0
            #plot.handles['y_range'].end = vals.max() * 1.1

        def temp_hook(plot, element):
            p = plot.state

            temp_color = 'red'

            y2_label = 'Temperature (°C)'
            # create secondary range and axis
            if 'twiny' not in [t for t in p.extra_y_ranges]:
                p.y_range = Range1d(start=plot.handles['y_range'].start,
                                    end=plot.handles['y_range'].end)
                p.y_range.name = 'default'
                p.extra_y_ranges = {"twiny": Range1d(start=-10, end=10)}
                p.add_layout(LinearAxis(axis_label=y2_label,
                                        y_range_name="twiny",
                                        axis_label_text_color=temp_color,
                                        major_label_text_color=temp_color,
                                        axis_line_color=temp_color,
                                        major_tick_line_color=temp_color,
                                        minor_tick_line_color=temp_color), 'right')

            # set glyph y_range_name to the one we've just created
            glyph = p.renderers[-1]
            glyph.y_range_name = 'twiny'
            vals = glyph.data_source.data['Temperature']

            # set range of second y axis
            y_add_range = (vals.max() - vals.min()) * 0.05
            p.extra_y_ranges["twiny"].start = vals.min() - y_add_range
            p.extra_y_ranges["twiny"].end = vals.max() + y_add_range

        if ref_height is None:
            ref_height = self.ref_hgt

        temp_data = []
        prcp_data = []
        for month in list(self.months.keys()):
            temp, tempformelt, prcp_total, prcp_sol, fac = \
                self.get_monthly_climate(heights=[ref_height], month=month)
            temp_data = np.append(temp_data, temp)
            prcp_data = np.append(prcp_data, prcp_total / self._prcp_fac)

        # this is needed to show Temperatuer and Precipitation in the same hover
        df = pd.DataFrame({'Month': np.array(list(self.months.keys())),
                           'MonthLong': np.array(['October',
                                                  'November',
                                                  'December',
                                                  'January',
                                                  'February',
                                                  'March',
                                                  'April',
                                                  'May',
                                                  'June',
                                                  'July',
                                                  'August',
                                                  'September']),
                           'Precipitation': np.array(prcp_data),
                           'Temperature': np.array(temp_data)})
        
        hover = HoverTool(tooltips="""
            <div>
                <span style="font-size: 12px; color: #000000;">Month: @{MonthLong}</span>
                <br />
                <span style="font-size: 12px; color: #0000FF;">Precipitation: @{Precipitation} mm</span>
                <br />
                <span style="font-size: 12px; color: #FF0000;">Temperature: @{Temperature}{%0.1f} °C</span>
            </div>""",
                          formatters={'@{Temperature}': 'printf'},
                          mode='vline')
        tools = ['save', hover]
        
        return (hv.Bars(df,
                        kdims='Month',
                        vdims=['Precipitation', 'Temperature', 'MonthLong'],
                        ).opts(color='blue',
                               responsive=True,
                               ylim=(0, max(prcp_data) * 1.1),
                               ylabel='Precipitation (mm)',
                               # min_height=300,
                               hooks=[prcp_hook],
                               xlabel='',
                               default_tools=tools
                               ) *
                hv.Curve(df,
                         vdims='Temperature',
                         kdims='Month'
                         ).opts(color='red',
                                hooks=[temp_hook],
                                responsive=True,
                                # min_height=300,
                                xlabel='',
                                default_tools=['save'] #tools
                                )
                ).opts(title='Climograph at ' + str(ref_height) + ' m',
                       xrotation=45)

    def get_xlim_annual(self, **kwargs):
        heights = self.default_heights
        t, tformelt, prcp, prcpsol, fac = self._get_2d_annual_climate(
            heights=heights)
        annual_ablation = - self.mu * np.sum(tformelt, axis=1) / self.rho
        annual_accumulation = np.sum(prcpsol, axis=1) / self.rho

        return (np.min(annual_ablation) - 5, np.max(annual_accumulation) + 5)

    def get_annual_mb_curve(self, **kwargs):
        heights = self.default_heights
        mb_annual = self.get_annual_mb(heigths=heights) * SEC_IN_YEAR

        annual_mb_curve = hv.Curve((mb_annual, heights),
                                   kdims='MB annual' + str(self.mb_model_nr),
                                   vdims='height (m)',
                                   label='annual').opts(responsive=True,
                                                        default_tools=self.default_tools,
                                                        active_tools=self.active_tools,
                                                        xlabel='mm w.e./year',
                                                        ylim=(
                                                            np.min(heights), np.max(heights)),
                                                        xlim=self.get_xlim_annual()
                                                        )

        annual_mb_curve.opts(opts.Curve(framewise=True))

        return annual_mb_curve

    def get_month_mb_curve(self, month=None, **kwargs):
        heights = self.default_heights
        mb_month = self.get_monthly_mb(
            heights=heights, month=month) * SEC_IN_MONTH

        month_mb_curve = hv.Curve((mb_month, heights),
                                  kdims='MB ' + month + str(self.mb_model_nr),
                                  vdims='height (m)',
                                  label=month
                                  ).opts(responsive=True,
                                         default_tools=self.default_tools,
                                         active_tools=self.active_tools,
                                         xlabel='mm w.e./month',
                                         ylim=(np.min(heights),
                                               np.max(heights)),
                                         xlim=(np.min(mb_month) - 1,
                                               np.max(mb_month) + 1)
                                         )

        month_mb_curve.opts(opts.Curve(framewise=True))

        return month_mb_curve

    def get_annual_accumulation_curve(self, **kwargs):
        heights = self.default_heights
        t, tformelt, prcp, prcpsol, fac = self._get_2d_annual_climate(
            heights=heights)
        annual_accumulation = np.sum(prcpsol, axis=1) / self.rho

        annual_accumulation_curve = hv.Curve((annual_accumulation, heights),
                                             kdims='Acc. annual' +
                                             str(self.mb_model_nr),
                                             vdims='height (m)',
                                             label='annual'
                                             ).opts(responsive=True,
                                                    default_tools=self.default_tools,
                                                    active_tools=self.active_tools,
                                                    xlabel='mm w.e./year',
                                                    ylim=(np.min(heights),
                                                          np.max(heights)),
                                                    xlim=self.get_xlim_annual()
                                                    )

        annual_accumulation_curve.opts(opts.Curve(framewise=True))
        return annual_accumulation_curve

    def get_annual_ablation_curve(self, **kwargs):
        heights = self.default_heights
        t, tformelt, prcp, prcpsol, fac = self._get_2d_annual_climate(
            heights=heights)
        annual_ablation = - self.mu * np.sum(tformelt, axis=1) / self.rho

        annual_ablation_curve = hv.Curve((annual_ablation, heights),
                                         kdims='Abl. annual' +
                                         str(self.mb_model_nr),
                                         vdims='height (m)',
                                         label='annual'
                                         ).opts(responsive=True,
                                                default_tools=self.default_tools,
                                                active_tools=self.active_tools,
                                                xlabel='mm w.e./year',
                                                ylim=(np.min(heights),
                                                      np.max(heights)),
                                                xlim=self.get_xlim_annual()
                                                )

        annual_ablation_curve.opts(opts.Curve(framewise=True))

        return annual_ablation_curve

    def get_month_accumulation_curve(self, month=None, **kwargs):
        heights = self.default_heights
        t, tformelt, prcp, prcpsol, fac = self.get_monthly_climate(
            heights=heights, month=month)
        month_accumulation = prcpsol / self.rho

        month_accumulation_curve = hv.Curve((month_accumulation, heights),
                                            kdims='Acc. ' + month +
                                            str(self.mb_model_nr),
                                            vdims='height (m)',
                                            label=month
                                            ).opts(responsive=True,
                                                   default_tools=self.default_tools,
                                                   active_tools=self.active_tools,
                                                   xlabel='mm w.e./month',
                                                   ylim=(np.min(heights),
                                                         np.max(heights)),
                                                   xlim=self.get_xlim_annual()
                                                   )

        month_accumulation_curve.opts(opts.Curve(framewise=True))

        return month_accumulation_curve

    def get_month_ablation_curve(self, month=None, **kwarg):
        heights = self.default_heights
        t, tformelt, prcp, prcpsol, fac = self.get_monthly_climate(
            heights=heights, month=month)
        month_ablation = - self.mu * tformelt / self.rho

        month_ablation_curve = hv.Curve((month_ablation, heights),
                                        kdims='Abl. ' + month +
                                        str(self.mb_model_nr),
                                        vdims='height (m)',
                                        label=month
                                        ).opts(responsive=True,
                                               default_tools=self.default_tools,
                                               active_tools=self.active_tools,
                                               xlabel='mm w.e./month',
                                               ylim=(np.min(heights),
                                                     np.max(heights)),
                                               xlim=self.get_xlim_annual()
                                               )

        month_ablation_curve.opts(opts.Curve(framewise=True))

        return month_ablation_curve

    def get_ELA_curve(self, kdims=None, **kwarg):
        heights = self.default_heights

        ELA = self.get_ela()

        ELA_curve = hv.Curve((list(self.get_xlim_annual()),
                              [ELA, ELA]),
                             kdims=kdims + str(self.mb_model_nr),
                             vdims='height (m)',
                             label='ELA'
                             ).opts(responsive=True,
                                    color='black',
                                    line_dash='dashed',
                                    default_tools=self.default_tools,
                                    active_tools=self.active_tools,
                                    ylim=(np.min(heights), np.max(heights)),
                                    xlim=self.get_xlim_annual()
                                    )

        ELA_curve.opts(opts.Curve(framewise=True))

        return ELA_curve

    def get_total_prcp_curve(self, month=None):
        heights = self.default_heights
        t, tformelt, prcp, prcpsol, fac = self.get_monthly_climate(
            heights=heights, month=month)

        if np.any(np.where(fac == 0)):
            height_t_liquid = heights[np.where(fac == 0)[0][-1]]
        else:
            height_t_liquid = None

        if np.any(np.where(fac == 1)):
            height_t_solid = heights[np.where(fac == 1)[0][0]]
        else:
            height_t_solid = None

        ELA = self.get_ela()

        prcp_xlim = (np.min(prcp) - 10, np.max(prcp) + 10)

        prcp_total_curve = (hv.Curve((prcp, heights),
                                     kdims='Precipitation (mm)' +
                                     str(self.mb_model_nr),
                                     vdims='height (m)',
                                     ).opts(responsive=True,
                                            default_tools=self.default_tools,
                                            active_tools=self.active_tools,
                                            ) *
                            hv.Curve((prcp_xlim, (ELA, ELA)),
                                     vdims='height(m)',
                                     kdims='ELA' + str(self.mb_model_nr),
                                     label='ELA'
                                     ).opts(responsive=True,
                                            color='black',
                                            line_dash='dashed',
                                            default_tools=self.default_tools,
                                            active_tools=self.active_tools,
                                            ) *
                            hv.Curve((prcp_xlim, (height_t_solid, height_t_solid)),
                                     vdims='height (m)',
                                     kdims='h(T_sol)' + str(self.mb_model_nr),
                                     label='h(T_solid)'
                                     ).opts(responsive=True,
                                            default_tools=self.default_tools,
                                            active_tools=self.active_tools,
                                            color='gray',
                                            line_width=2,
                                            line_alpha=0.5,
                                            line_dash='dotted'
                                            ) *
                            hv.Curve((prcp_xlim, (height_t_liquid, height_t_liquid)),
                                     vdims='height (m)',
                                     kdims='h(T_liq)' + str(self.mb_model_nr),
                                     label='h(T_liquid)'
                                     ).opts(responsive=True,
                                            default_tools=self.default_tools,
                                            active_tools=self.active_tools,
                                            color='gray',
                                            line_width=2,
                                            line_alpha=0.5,
                                            line_dash='dotdash')
                            ).opts(title='Prcp_total',
                                   toolbar=None,
                                   xlim=prcp_xlim,
                                   xlabel='Precipitation (mm)',
                                   legend_position='top_left',
                                   ylim=(np.min(heights), np.max(heights)),
                                   responsive=True
                                   )
        prcp_total_curve.opts(opts.Curve(framewise=True))

        return prcp_total_curve

    def get_f_curve(self, month=None):
        def f_hook(plot, state):
            p = plot.state
            p.yaxis.major_label_text_font_size = '0pt'
            p.yaxis.axis_label_text_font_size = '0pt'

        heights = self.default_heights
        t, tformelt, prcp, prcpsol, fac = self.get_monthly_climate(
            heights=heights, month=month)

        if np.any(np.where(fac == 0)):
            height_t_liquid = heights[np.where(fac == 0)[0][-1]]
        else:
            height_t_liquid = None

        if np.any(np.where(fac == 1)):
            height_t_solid = heights[np.where(fac == 1)[0][0]]
        else:
            height_t_solid = None

        ELA = self.get_ela()

        f_xlim = (-0.2, 1.2)

        f_curve = (hv.Curve((fac, heights),
                            kdims='f' + str(self.mb_model_nr),
                            vdims='height (m)',
                            ).opts(responsive=True,
                                   default_tools=self.default_tools,
                                   active_tools=self.active_tools,
                                   ) *
                   hv.Curve((f_xlim, (ELA, ELA)),
                            vdims='height(m)',
                            kdims='ELA' + str(self.mb_model_nr),
                            ).opts(responsive=True,
                                   color='black',
                                   line_dash='dashed',
                                   default_tools=self.default_tools,
                                   active_tools=self.active_tools
                                   ) *
                   hv.Curve((f_xlim, (height_t_solid, height_t_solid)),
                            vdims='height (m)',
                            kdims='h(T_sol)' + str(self.mb_model_nr),
                            ).opts(responsive=True,
                                   default_tools=self.default_tools,
                                   active_tools=self.active_tools,
                                   color='gray',
                                   line_width=2,
                                   line_alpha=0.5,
                                   line_dash='dotted') *
                   hv.Curve((f_xlim, (height_t_liquid, height_t_liquid)),
                            vdims='height (m)',
                            kdims='h(T_liq)' + str(self.mb_model_nr),
                            ).opts(responsive=True,
                                   default_tools=self.default_tools,
                                   active_tools=self.active_tools,
                                   color='gray',
                                   line_width=2,
                                   line_alpha=0.5,
                                   line_dash='dotdash')
                   ).opts(width=40,
                          toolbar=None,
                          title='f',
                          ylabel='',
                          xlabel='f',
                          xlim=f_xlim,
                          xticks=[0, 1],
                          hooks=[f_hook],
                          ylim=(np.min(heights), np.max(heights)),
                          responsive=True
                          )

        return f_curve

    def get_t_curve(self, month=None):
        heights = self.default_heights

        t, tformelt, prcp, prcpsol, fac = self.get_monthly_climate(
            heights=heights, month=month)

        ELA = self.get_ela()

        t_xlim = (np.min(np.concatenate((t, tformelt))) - 3,
                  np.max(np.concatenate((t, tformelt))) + 3)

        t_curve = (hv.VLine(0).opts(color='gray',
                                    default_tools=self.default_tools,
                                    active_tools=self.active_tools,
                                    line_width=1,
                                    line_alpha=0.5
                                    ) *
                   hv.Curve((t_xlim, (ELA, ELA)),
                            vdims='height(m)',
                            kdims='ELA' + str(self.mb_model_nr),
                            ).opts(responsive=True,
                                   color='black',
                                   line_dash='dashed',
                                   default_tools=self.default_tools,
                                   active_tools=self.active_tools
                                   ) *
                   hv.Curve((t, heights),
                            kdims='Temperatuer (°C)' + str(self.mb_model_nr),
                            vdims='height (m)',
                            label='Temperature T'
                            ).opts(responsive=True,
                                   default_tools=self.default_tools,
                                   active_tools=self.active_tools,
                                   ) *
                   hv.Curve((tformelt, heights),
                            kdims='T_for_melt' + str(self.mb_model_nr),
                            vdims='height (m)',
                            label=r'max(T - T_melt, 0)'
                            ).opts(responsive=True,
                                   default_tools=self.default_tools,
                                   active_tools=self.active_tools,
                                   )
                   ).opts(toolbar=None,
                          ylim=(np.min(heights), np.max(heights)),
                          xlabel='Temperature (°C)',
                          title='T for melt',
                          ylabel='',
                          xlim=t_xlim
                          )

        t_curve.opts(opts.Curve(framewise=True))
        return t_curve

# Define default MassBalanceModels

In [ ]:
mb_models = [OGGMMassBalance(min_height=min_height,
                             max_height=max_height,
                             temp=base_climate_temps[0],
                             prcp=base_climate_prcps[0],
                             mb_model_nr=0,
                             base_climate_name=base_climate_names[0],
                             ref_hgt=base_climate_ref_hgts[0],),
             OGGMMassBalance(min_height=min_height,
                             max_height=max_height,
                             temp=base_climate_temps[1],
                             prcp=base_climate_prcps[1],
                             mb_model_nr=1,
                             base_climate_name=base_climate_names[1],
                             ref_hgt=base_climate_ref_hgts[1],), ]

current_mb_i = 0

# Connect MB Models with menu

## climate

In [ ]:
def set_climate_of_mb_model(mb_i):
    global mb_models

    def set_base_climate(mb_i, base_climate_i):
        mb_models[mb_i].temp = base_climate_temps[base_climate_i]
        mb_models[mb_i].prcp = base_climate_prcps[base_climate_i]
        mb_models[mb_i].ref_hgt = base_climate_ref_hgts[base_climate_i]
        mb_models[mb_i].base_climate_name = base_climate_names[base_climate_i]
    if base_climate_select.value == base_climate_names[0]:
        set_base_climate(mb_i, 0)
    elif base_climate_select.value == base_climate_names[1]:
        set_base_climate(mb_i, 1)

    mb_models[mb_i].ref_hgt = np.array(float(ref_height_slider.value))
    mb_models[mb_i].temp_bias = np.array(float(temp_bias_slider.value))
    mb_models[mb_i].prcp_fac = np.array(float(prcp_fac_slider.value))
    mb_models[mb_i].temp_grad = np.array(float(temp_grad_slider.value)) / 100
    mb_models[mb_i].prcp_grad = np.array(float(prcp_grad_slider.value))

In [ ]:
def set_climate_menu_with_selected_mb_model(mb_i):
    base_climate_select.value = mb_models[mb_i].base_climate_name
    ref_height_slider.value = int(mb_models[mb_i].ref_hgt)
    temp_bias_slider.value = float(mb_models[mb_i].temp_bias)
    prcp_fac_slider.value = float(mb_models[mb_i].prcp_fac)
    temp_grad_slider.value = float(mb_models[mb_i].temp_grad) * 100
    prcp_grad_slider.value = float(mb_models[mb_i].prcp_grad)

## MB settings

In [ ]:
def set_mb_settings_of_mb_model(mb_i):
    mb_models[mb_i].mu = np.array(mu_slider.value)
    mb_models[mb_i].t_melt = np.array(temp_melt_slider.value)
    mb_models[mb_i].t_solid = np.array(temp_solid_slider.value)
    mb_models[mb_i].t_liq = np.array(temp_liquid_slider.value)

In [ ]:
def set_mb_settings_menu_with_selected_mb_model(mb_i):
    mu_slider.value = int(mb_models[mb_i].mu)
    temp_melt_slider.value = float(mb_models[mb_i].t_melt)
    temp_solid_slider.value = float(mb_models[mb_i].t_solid)
    temp_liquid_slider.value = float(mb_models[mb_i].t_liq)

# Detailed plots

In [ ]:
def toogle_detailed_disabled():
    select_months_detailed_0.disabled = not select_months_detailed_0.disabled
    select_months_detailed_1.disabled = not select_months_detailed_1.disabled

In [ ]:
def dyn_total_prcp_curve(month, update, mb_i=0):
    return mb_models[mb_i].get_total_prcp_curve(month=month)


def dyn_f_curve(month, update, mb_i=0):
    return mb_models[mb_i].get_f_curve(month=month)


def dyn_t_curve(month, update, mb_i=0):
    return mb_models[mb_i].get_t_curve(month=month)


def dyn_month_mb_curve(month, update, mb_i=0):
    return (mb_models[mb_i].get_month_mb_curve(month=month) *
            mb_models[mb_i].get_ELA_curve(kdims='').opts(xlim=(None, None)) *
            hv.VLine(0).opts(color='gray',
                             default_tools=mb_models[mb_i].default_tools,
                             active_tools=mb_models[mb_i].active_tools,
                             line_width=1,
                             line_alpha=0.5
                             )
            ).opts(title='Total Mass Balance ' + month,
                   ylabel='',
                   toolbar=None,
                   show_legend=False)


select_months_detailed_0 = pn.widgets.RadioBoxGroup(
    name='month selection 0',
    options=list(mb_models[0].months.keys()),
    inline=False,
    width=50)
select_months_detailed_1 = pn.widgets.RadioBoxGroup(
    name='month selection 1',
    options=list(mb_models[1].months.keys()),
    inline=False,
    width=50)
select_months_detailed_both = [select_months_detailed_0,
                               select_months_detailed_1]

detailed_updater_0 = pn.widgets.Checkbox(name='Update Detailed 0')
detailed_updater_1 = pn.widgets.Checkbox(name='Update Detailed 1')
detailed_updater_both = [detailed_updater_0,
                         detailed_updater_1]

dmap_total_prcp_both = [hv.DynamicMap(
    pn.bind(partial(dyn_total_prcp_curve, mb_i=0),
            month=select_months_detailed_0,
            update=detailed_updater_0)),
    hv.DynamicMap(
    pn.bind(partial(dyn_total_prcp_curve, mb_i=1),
            month=select_months_detailed_1,
            update=detailed_updater_1)), ]

dmap_f_curve_both = [hv.DynamicMap(
    pn.bind(partial(dyn_f_curve, mb_i=0),
            month=select_months_detailed_0,
            update=detailed_updater_0)),
    hv.DynamicMap(
    pn.bind(partial(dyn_f_curve, mb_i=1),
            month=select_months_detailed_1,
            update=detailed_updater_1)), ]

dmap_t_curve_both = [hv.DynamicMap(
    pn.bind(partial(dyn_t_curve, mb_i=0),
            month=select_months_detailed_0,
            update=detailed_updater_0)),
    hv.DynamicMap(
    pn.bind(partial(dyn_t_curve, mb_i=1),
            month=select_months_detailed_1,
            update=detailed_updater_1)), ]

dmap_month_mb_curve_both = [hv.DynamicMap(
    pn.bind(partial(dyn_month_mb_curve, mb_i=0),
            month=select_months_detailed_0,
            update=detailed_updater_0)),
    hv.DynamicMap(
    pn.bind(partial(dyn_month_mb_curve, mb_i=1),
            month=select_months_detailed_1,
            update=detailed_updater_1)), ]


def get_complete_detailed_figure(mb_i=0):
    return pn.Row(pn.Column(pn.layout.VSpacer(),
                            select_months_detailed_both[mb_i],
                            pn.layout.VSpacer(),
                            width=50,
                            sizing_mode="stretch_height",
                            ),
                  pn.Row(pn.Row(dmap_total_prcp_both[mb_i],
                                get_simple_text(text='*',
                                                width=15,
                                                font_size='15pt'),
                                dmap_f_curve_both[mb_i],
                                get_simple_text(text='/',
                                                width=15,
                                                font_size='15pt'),
                                get_latex_text(text=r'$\rho_{ice}$',
                                               width=30,
                                               font_size='15pt'),
                                sizing_mode='stretch_both'
                                ),
                         get_simple_text('+'),
                         pn.Row(get_latex_text(text=r'(-$\mu$)',
                                               width=30,
                                               font_size='15pt'),
                                get_simple_text(text='*',
                                                width=15,
                                                font_size='15pt'),
                                dmap_t_curve_both[mb_i],
                                sizing_mode='stretch_both'),
                         get_simple_text('='),
                         dmap_month_mb_curve_both[mb_i],
                         sizing_mode='stretch_both',
                         ),
                  sizing_mode='stretch_both',
                  # height=400,
                  )


detailed_figures = [get_complete_detailed_figure(0),
                    get_complete_detailed_figure(1)]

# Coarse plots

In [ ]:
def dyn_overlay_accumulation(months, update, mb_i=0):
    overlay = mb_models[mb_i].get_ELA_curve(kdims='Acc.')
    overlay *= hv.VLine(0).opts(color='gray',
                                line_width=1,
                                line_alpha=0.5,
                                default_tools=mb_models[mb_i].default_tools,
                                active_tools=mb_models[mb_i].active_tools,
                                )
    overlay *= mb_models[mb_i].get_annual_accumulation_curve()
    for month in list(mb_models[mb_i].months.keys()):
        if month in months:
            overlay *= mb_models[mb_i].get_month_accumulation_curve(
                month=month)
        else:
            overlay *= hv.Curve((0, 0), label='').opts(responsive=True,
                                                       default_tools=mb_models[mb_i].default_tools,
                                                       active_tools=mb_models[mb_i].active_tools,
                                                       xlabel='mm w.e./month',
                                                       ylim=(np.min(mb_models[mb_i].default_heights),
                                                             np.max(mb_models[mb_i].default_heights))
                                                       )
    return overlay.opts(title='Accumulation',
                        toolbar=None,
                        show_legend=True,
                        legend_position='top_left',
                        )


def dyn_overlay_ablation(months, update, mb_i=0):
    overlay = mb_models[mb_i].get_ELA_curve(kdims='Abl.')
    overlay *= hv.VLine(0).opts(color='gray',
                                line_width=1,
                                line_alpha=0.5,
                                default_tools=mb_models[mb_i].default_tools,
                                active_tools=mb_models[mb_i].active_tools,
                                )
    overlay *= mb_models[mb_i].get_annual_ablation_curve()
    for month in list(mb_models[mb_i].months.keys()):
        if month in months:
            overlay *= mb_models[mb_i].get_month_ablation_curve(month=month)
        else:
            overlay *= hv.Curve((0, 0), label='').opts(responsive=True,
                                                       default_tools=mb_models[mb_i].default_tools,
                                                       active_tools=mb_models[mb_i].active_tools,
                                                       xlabel='mm w.e./month',
                                                       ylim=(np.min(mb_models[mb_i].default_heights),
                                                             np.max(mb_models[mb_i].default_heights))
                                                       )
    return overlay.opts(title='Ablation',
                        ylabel='',
                        toolbar=None,
                        show_legend=False,
                        )


def dyn_overlay_total_mb(months, update, mb_i=0):
    overlay = mb_models[mb_i].get_ELA_curve(kdims='MB')
    overlay *= hv.VLine(0).opts(color='gray',
                                line_width=1,
                                line_alpha=0.5,
                                default_tools=mb_models[mb_i].default_tools,
                                active_tools=mb_models[mb_i].active_tools,
                                )
    overlay *= mb_models[mb_i].get_annual_mb_curve()
    for month in list(mb_models[mb_i].months.keys()):
        if month in months:
            overlay *= mb_models[mb_i].get_month_mb_curve(
                month=month).opts(xlim=(None, None))
        else:
            overlay *= hv.Curve((0, 0), label='').opts(responsive=True,
                                                       default_tools=mb_models[mb_i].default_tools,
                                                       active_tools=mb_models[mb_i].active_tools,
                                                       xlabel='mm w.e./month',
                                                       ylim=(np.min(mb_models[mb_i].default_heights),
                                                             np.max(mb_models[mb_i].default_heights))
                                                       )
    return overlay.opts(title='Total Mass-Balance',
                        ylabel='',
                        toolbar=None,
                        show_legend=False,
                        )


select_months_coarse_0 = pn.widgets.CheckBoxGroup(
    name='month selection',
    value=[],
    options=list(mb_models[0].months.keys()),
    inline=False,
    width=50)
select_months_coarse_1 = pn.widgets.CheckBoxGroup(
    name='month selection',
    value=[],
    options=list(mb_models[1].months.keys()),
    inline=False,
    width=50)
select_months_coarse_both = [select_months_coarse_0,
                             select_months_coarse_1]

coarse_updater_0 = pn.widgets.Checkbox(name='Update Coarse 0')
coarse_updater_1 = pn.widgets.Checkbox(name='Update Coarse 1')
coarse_updater_both = [coarse_updater_0,
                         coarse_updater_1]

dmap_accumulation_both = [hv.DynamicMap(
    pn.bind(partial(dyn_overlay_accumulation, mb_i=0),
            months=select_months_coarse_0,
            update=coarse_updater_0)),
    hv.DynamicMap(
    pn.bind(partial(dyn_overlay_accumulation, mb_i=1),
            months=select_months_coarse_1,
            update=coarse_updater_1)), ]

dmap_ablation_both = [hv.DynamicMap(
    pn.bind(partial(dyn_overlay_ablation, mb_i=0),
            months=select_months_coarse_0,
            update=coarse_updater_0)),
    hv.DynamicMap(
    pn.bind(partial(dyn_overlay_ablation, mb_i=1),
            months=select_months_coarse_1,
            update=coarse_updater_1)), ]

dmap_total_mb_both = [hv.DynamicMap(
    pn.bind(partial(dyn_overlay_total_mb, mb_i=0),
            months=select_months_coarse_0,
            update=coarse_updater_0)),
    hv.DynamicMap(
    pn.bind(partial(dyn_overlay_total_mb, mb_i=1),
            months=select_months_coarse_1,
            update=coarse_updater_1)), ]


def get_complete_coarse_figure(mb_i):
    return pn.Row(pn.Column(pn.layout.VSpacer(),
                            select_months_coarse_both[mb_i],
                            pn.layout.VSpacer(),
                            width=50,
                            sizing_mode="stretch_height",
                            ),
                  pn.Row(dmap_accumulation_both[mb_i],
                         get_simple_text('+'),
                         dmap_ablation_both[mb_i],
                         get_simple_text('='),
                         dmap_total_mb_both[mb_i],
                         sizing_mode='stretch_both'
                         ),
                  sizing_mode='stretch_both',
                  )


coarse_figures = [get_complete_coarse_figure(0),
                  get_complete_coarse_figure(1)]

In [ ]:
def toogle_coarse_disabled():
    select_months_coarse_0.disabled = not select_months_coarse_0.disabled
    select_months_coarse_1.disabled = not select_months_coarse_1.disabled

# Climographs

In [ ]:
def get_climograph(change, mb_i=0):
    return mb_models[mb_i].get_climograph().opts(toolbar=None,
                                                 height=250)

climograph_updater_0 = pn.widgets.Checkbox(name='Update Climograph 0')
climograph_updater_1 = pn.widgets.Checkbox(name='Update Climograph 1')
climograph_updater_both = [climograph_updater_0,
                           climograph_updater_1]

climographs = [hv.DynamicMap(
    pn.bind(partial(get_climograph, mb_i=0),
            change=climograph_updater_0)),
               hv.DynamicMap(
    pn.bind(partial(get_climograph, mb_i=1),
            change=climograph_updater_1)),]

# Function to toogle all things during work

In [ ]:
def toogle_work_status():
    toogle_busy_indicators()
    toogle_disabled_of_menu()
    toogle_detailed_disabled()
    toogle_coarse_disabled()

# Function to update figures

In [ ]:
def update_current_figures(mb_i):
    climograph_updater_both[mb_i].value = not climograph_updater_both[mb_i].value
    detailed_updater_both[mb_i].value = not detailed_updater_both[mb_i].value
    coarse_updater_both[mb_i].value = not coarse_updater_both[mb_i].value

# Define Equations

## MB equation

In [ ]:
mb_equation_font_size = '10pt'
mb_equation_margin = 5
mb_equation = pn.Column(pn.pane.LaTeX(r'''
$\begin{aligned}
  ~~~~MB_{annual} ~~ &=  \\
  =~~Accumulation_{annual}~~~~~~~~~~~~~~~~~~~~~~~~~~~+~~Ablation_{annual}~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~&= \\
  =~~\sum_{month=1}^{12} MB_{month}~~&= \\
  =~~\sum_{month=1}^{12} Accumulation_{month}~~~~~~~~~~+~~\sum_{month=1}^{12} Ablation_{month}~~~~~~~~~~~~~~~~~~~~~~~~~~~~~&= \\
  =~~\sum_{month=1}^{12}\left[ Prcp_{month} * f_{month} / \rho_{ice}\right]~~+~~\sum_{month=1}^{12}\left[(-\mu) * \mathrm{max}(T_{month} - T_{melt},0)\right]&
\end{aligned}
$''',
                                      style={
                                          'font-size': mb_equation_font_size},
                                      margin=(mb_equation_margin, mb_equation_margin)),
                        width=615,
                        #height=110,
                        )

## f equation

In [ ]:
f_equation_font_size = '10pt'
f_equation = pn.Column(pn.pane.LaTeX(r'''
$f_{month} =
\begin{cases}
1~ \mathrm{if}~T_{month} <= T_{solid}\\ 
0~ \mathrm{if}~T_{month} >= T_{liquid}\\
\mathrm{linearly~interpolated~in~between}
\end{cases}$
''',
                                     style={'font-size': f_equation_font_size}),
                       sizing_mode='stretch_width',
                       height=80,
                       margin=(0, 0)
                       )

# Text field for current MB settings

In [ ]:
current_mb_settings_text = pn.Column(pn.pane.Markdown(object=('<p style="margin-top: 0px;">' +
                                                              'TODO: Show the current Settings here (e.g. mu, T_grad, T_melt, ...)'
                                                              + '</p>'),
                                                      # height=20,
                                                      margin=(0, 0),
                                                      align='end',),
                                     height=200,
                                     sizing_mode='stretch_width')

# Put figures together

## help function

In [ ]:
def get_mb_tab(i):
    global mb_models

    # grid = pn.GridSpec(sizing_mode='stretch_both',
    #                   mode='override')
    #grid[0, 0] = climographs[i]
    # grid[0, 1:3] = pn.Column(current_mb_settings_text,
    #                         pn.Card(mb_equation,
    #                                 pn.layout.Divider(),
    #                                 f_equation,
    #                                 title='Mass Balance Model Equations',
    #                                 collapsed=True),
    #                         background='whitesmoke',
    #                         sizing_mode='stretch_height')
    #grid[1, :] = coarse_figures[i]
    return pn.Column(pn.Row(climographs[i],
                            pn.Tabs(('Current Mass Balance Settings',
                                     current_mb_settings_text),
                                    ('Mass Balance Model Equations',
                                     pn.Column(mb_equation,
                                               pn.layout.Divider(),
                                               f_equation,)),
                                    tabs_location='below',
                                    height=250,
                                    width=615,
                                   ),
                            sizing_mode='stretch_width'),
                     coarse_figures[i],
                     pn.Card(detailed_figures[i],
                             title='Detailed Mass-Balance for one Month',
                             collapsed=True,
                             height=500,
                             sizing_mode='stretch_width'
                             ),
                     sizing_mode='stretch_both')

## create figures tab

In [ ]:
def change_figures_tab(event):
    global current_mb_i

    if event.new == 0:
        # if base_climate_select.disabled:
        #    toogle_disabled_of_menu()
        current_mb_i = 0
        set_climate_menu_with_selected_mb_model(0)
        set_mb_settings_menu_with_selected_mb_model(0)

    elif event.new == 1:
        # if base_climate_select.disabled:
        #    toogle_disabled_of_menu()
        current_mb_i = 1
        set_climate_menu_with_selected_mb_model(1)
        set_mb_settings_menu_with_selected_mb_model(1)

    elif event.new == 2:
        pass
        # if not base_climate_select.disabled:
        #    toogle_disabled_of_menu()


figures_tab = pn.Tabs(('Mass Balance Model 1', get_mb_tab(0)),
                      ('Mass Balance Model 2', get_mb_tab(1)),
                      ('Compare Mass Balance Models', pn.Column(
                          pn.pane.Markdown(object=('<p style="margin-top: 0px;">' +
                                                   'TODO...'
                                                   + '</p>'),
                                           # height=20,
                                           margin=(0, 0),
                                           align='end',),
                                                              )
                      ),
                     )
figures_tab.param.watch(change_figures_tab, ['active']);

# Put App together

In [ ]:
initial_run = False

In [ ]:
app = pn.Row(tab_menu,
             figures_tab,
             sizing_mode='stretch_both')

In [ ]:
app.servable(title='Mass Balance Simulator')